# Calculate reproduction rate (R0) for infectious diseases within a micro-environment
This model calculates the reproduction rate (R0) of infectious diseases (Covid19) within a micro-environment based upon dispersion of an aerosol within an enclosed space. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individual's probability of being infected.

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [1]:
import math
import plotly.graph_objects as go
import simpy
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm, tnrange

# Import local libraries
from Simulation import Simulation

In [2]:
def run_simulation(simulation_name):
    periods = 180
    
    simulation_run = 1
    arrivals_per_hour=None # Get arrival rate from configuration
    max_arrivals=None
    quanta_emission_rate=147
    inhalation_rate=0.54

    simulation = Simulation(simulation_name, simulation_run, microenvironment=simulation_name, periods=periods)

    simulation.run(quanta_emission_rate=quanta_emission_rate, 
                    inhalation_rate=inhalation_rate, 
                    report_time=False)

    infections = simulation.get_counter('Infections')
    infections = infections if infections else 0

    total_visitors = simulation.get_counter('Total visitors')

    attack_rate = infections / total_visitors

    return infections, total_visitors, attack_rate

In [14]:
results = []
simulation_name = 'Restaurant-natural-No Lockdown' if True else 'Pharmacy-mechanical-Lockdown'
for i in trange(100):
    results.append(run_simulation(simulation_name))

In [15]:
import plotly.express as px
df = pd.DataFrame(results, columns=["Infections", "Total visitors", "Attack rate"])
fig = px.histogram(df, x="Infections")
fig.update_layout(
    title=simulation_name,
    xaxis_title="Reproduction rate (number of infections)",
    yaxis_title="Count of simulations")
fig.show()
df.mean()

Infections         43.690000
Total visitors    160.000000
Attack rate         0.273062
dtype: float64

In [9]:
file_db = pd.read_excel('./Configuration/Environment database.xlsx', header=4, engine='openpyxl')
environments = file_db['environment']

In [10]:
environments

0           Pharmacy-natural-No Lockdown
1        Pharmacy-mechanical-No Lockdown
2              Pharmacy-natural-Lockdown
3           Pharmacy-mechanical-Lockdown
4        Supermarket-natural-No Lockdown
5     Supermarket-mechanical-No Lockdown
6           Supermarket-natural-Lockdown
7        Supermarket-mechanical-Lockdown
8         Restaurant-natural-No Lockdown
9      Restaurant-mechanical-No Lockdown
10       Post Office-natural-No Lockdown
11    Post Office-mechanical-No Lockdown
12          Post Office-natural-Lockdown
13       Post Office-mechanical-Lockdown
14              Bank-natural-No Lockdown
15           Bank-mechanical-No Lockdown
16                 Bank-natural-Lockdown
17              Bank-mechanical-Lockdown
18      Lounge-Two People-Winter-15 Mins
19      Lounge-Two People-Winter-30 Mins
20       Lounge-Two People-Winter-1 hour
21      Lounge-Two People-Summer-15 Mins
22      Lounge-Two People-Summer-30 Mins
23       Lounge-Two People-Summer-1 hour
24     Lounge-Fo

In [8]:
import multiprocess as mp
from run_parallel_simulation import run_parallel_simulation
sim_results = {}

for _, environment_name in environments.items():

    results = []
    #for i in tnrange(1000, desc=environment_name, leave=True):
    #    results.append(run_simulation(environment_name)) 

    max_iter = 1000
    with mp.Pool(max(1, int(os.cpu_count()*2/3-1))) as pool:
        results = list(tqdm(pool.imap(run_parallel_simulation, iter([environment_name] * max_iter)), desc=environment_name, total=max_iter))

    df = pd.DataFrame(results, columns=["Infections", "Total visitors", "Attack rate"])

    infections_mean = df.mean()['Infections']
    infections_sd = df.std()['Infections']
    total_visitors_mean = df.mean()['Total visitors']
    total_visitors_sd = df.std()['Total visitors']
    attack_rate = infections_mean / total_visitors_mean

    sim_results[environment_name] = (infections_mean, infections_sd, total_visitors_mean, total_visitors_sd, attack_rate)


In [9]:
df_results = pd.DataFrame.from_dict(sim_results, orient='index', columns=['infections mean', 'infections sd','visitors mean', 'visitors sd', 'attack rate'])
df_results

,infections mean,infections sd,visitors mean,visitors sd,attack rate
Pharmacy-natural-No Lockdown,4.577,2.204752,180.0,0.0,0.025428
Pharmacy-mechanical-No Lockdown,0.966,0.972517,180.0,0.0,0.005367
Pharmacy-natural-Lockdown,2.442,1.512258,90.0,0.0,0.027133
Pharmacy-mechanical-Lockdown,0.466,0.699524,90.0,0.0,0.005178
Supermarket-natural-No Lockdown,2.910,1.685450,360.0,0.0,0.008083
Supermarket-mechanical-No Lockdown,1.124,1.038124,360.0,0.0,0.003122
Supermarket-natural-Lockdown,0.466,0.661272,60.0,0.0,0.007767
Supermarket-mechanical-Lockdown,0.208,0.450487,60.0,0.0,0.003467
Restaurant-natural-No Lockdown,43.243,5.203752,160.0,0.0,0.270269
Restaurant-mechanical-No Lockdown,1.921,1.361107,160.0,0.0,0.012006


In [10]:
fig = px.scatter(df_results, x="infections mean", y="attack rate", hover_data=[df_results.index])
fig.update_layout(
    title="Infections vs attack rate",
    xaxis_title="Infections (mean)",
    yaxis_title="Attack rate")
fig.show()

In [11]:
df_results.to_excel('./output/environment reproduction rates.xlsx', 'reproduction rates')